In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plot
import numpy as np
import google.datalab.bigquery as bq

In [16]:
%%bq query --name os_stat
SELECT time, cpu_usage, max_cpu_usage, free_memory, min_free_memory, total_memory, (total_memory - free_memory)/total_memory as use_memory_rate
FROM `exem-191100.ods_im5.xapm_db_os_stat`
WHERE db_id = 6
ORDER BY time

In [57]:
%%bq sample --query os_stat --count 200000

time,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate
2017-11-03 15:00:00,5244,6670,246592,241860,6117496,0.959690696978
2017-11-03 15:01:00,5201,5720,246007,241844,6117496,0.959786324339
2017-11-03 15:02:00,5168,5460,245579,241372,6117496,0.959856287605
2017-11-03 15:03:00,5157,5330,245377,241372,6117496,0.959889307651
2017-11-03 15:04:00,5263,5610,242621,240992,6117496,0.960339818776
2017-11-03 15:05:00,5220,5600,244343,240256,6117496,0.960058331056
2017-11-03 15:06:00,5218,5530,243530,239876,6117496,0.960191228568
2017-11-03 15:07:00,5274,5630,243200,239636,6117496,0.960245172208
2017-11-03 15:08:00,5142,5430,242762,239132,6117496,0.960316770129
2017-11-03 15:09:00,5260,5520,242303,238636,6117496,0.960391800828


# os_stat table에는 있는데, stat table에는 없는 시간들..

In [11]:
%%bq query --name os_stat_only
SELECT time, cpu_usage, max_cpu_usage, free_memory, min_free_memory, total_memory, (total_memory - free_memory)/total_memory as use_memory_rate
FROM `exem-191100.ods_im5.xapm_db_os_stat` as t_os_stat
WHERE db_id = 6 and NOT EXISTS(
  SELECT time, stat_id, value
  FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat
  WHERE db_id = 6 and EXISTS(
    SELECT stat_id
    FROM(
      SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value
      FROM `exem-191100.ods_im5.xapm_db_stat`
      WHERE db_id = 6
      GROUP BY stat_id
      HAVING max_value <> 0  and min_value <> 0
      ORDER BY stat_id) as t_nonzero
    WHERE t_stat.stat_id = t_nonzero.stat_id
    ) and stat_id<687
      and t_os_stat.time = t_stat.time
  )
ORDER BY time

In [12]:
%%bq sample --query os_stat_only --count 200000

time,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate
2017-12-02 22:53:00,5151,5410,288856,284848,6117496,0.952781987924
2017-12-02 22:55:00,5341,5750,288408,284848,6117496,0.952855220502
2017-12-02 22:56:00,5305,5530,288074,284584,6117496,0.952909818004
2017-12-02 23:08:00,5169,5490,283227,281220,6117496,0.953702135645
2017-12-02 23:11:00,5206,5650,287051,283360,6117496,0.953077043287


# stat table에는 있는데, os_stat table에는 없는 시간들..

In [2]:
%%bq query --name stat_only
SELECT time, COUNT(stat_id)
FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat
WHERE db_id = 6 and NOT EXISTS(
  SELECT time
  FROM `exem-191100.ods_im5.xapm_db_os_stat` as t_os_stat
  WHERE db_id = 6 and t_stat.time = t_os_stat.time
  )
  and EXISTS(
    SELECT stat_id
    FROM(
      SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value
      FROM `exem-191100.ods_im5.xapm_db_stat`
      WHERE db_id = 6
      GROUP BY stat_id
      HAVING max_value <> 0  and min_value <> 0
      ORDER BY stat_id) as t_nonzero
    WHERE t_stat.stat_id = t_nonzero.stat_id
    ) 
  and stat_id<687
GROUP BY time
ORDER BY time

In [3]:
%%bq sample --query stat_only

time,f0_
2017-12-04 11:20:00,34
2017-12-04 11:21:00,34


# timestamp 의 빵구를 찾기위해서 시작시간 ~ 끝시간 까지의 시간 테이블을 Dataframe을 이용하여 만들자

## os_stat table의 time 시작과 끝

In [15]:
%%bq query --name os_stat_unix_second
SELECT UNIX_SECONDS(time) as unix_second, cpu_usage, max_cpu_usage, free_memory, min_free_memory, total_memory, (total_memory - free_memory)/total_memory as use_memory_rate
FROM `exem-191100.ods_im5.xapm_db_os_stat`
WHERE db_id = 6
ORDER BY time

In [16]:
%%bq sample --query os_stat_unix_second --count 200000

unix_second,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate
1509721200,5244,6670,246592,241860,6117496,0.959690696978
1509721260,5201,5720,246007,241844,6117496,0.959786324339
1509721320,5168,5460,245579,241372,6117496,0.959856287605
1509721380,5157,5330,245377,241372,6117496,0.959889307651
1509721440,5263,5610,242621,240992,6117496,0.960339818776
1509721500,5220,5600,244343,240256,6117496,0.960058331056
1509721560,5218,5530,243530,239876,6117496,0.960191228568
1509721620,5274,5630,243200,239636,6117496,0.960245172208
1509721680,5142,5430,242762,239132,6117496,0.960316770129
1509721740,5260,5520,242303,238636,6117496,0.960391800828


In [97]:
os_stat_unix_second_sql = 'SELECT UNIX_SECONDS(time) as unix_second FROM `exem-191100.ods_im5.xapm_db_os_stat` WHERE db_id = 6 ORDER BY time'

os_stat_unix_second_query = bq.Query(os_stat_unix_second_sql)

os_stat_unix_second_df = os_stat_unix_second_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [198]:
print(os_stat_unix_second_df.head(10))

   unix_second
0   1509721200
1   1509721260
2   1509721320
3   1509721380
4   1509721440
5   1509721500
6   1509721560
7   1509721620
8   1509721680
9   1509721740


In [17]:
%%bq query --name os_stat_unix_second_descending
SELECT UNIX_SECONDS(time) as unix_second, cpu_usage, max_cpu_usage, free_memory, min_free_memory, total_memory, (total_memory - free_memory)/total_memory as use_memory_rate
FROM `exem-191100.ods_im5.xapm_db_os_stat`
WHERE db_id = 6
ORDER BY time DESC

In [18]:
%%bq sample --query os_stat_unix_second_descending --count 200000

unix_second,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate
1512386340,5257,5690,288554,285708,6117496,0.952831354528
1512386280,5138,5430,289245,285584,6117496,0.952718399816
1512386220,5164,5410,289808,286212,6117496,0.952626368697
1512386160,5177,5470,289573,285716,6117496,0.952664783107
1512386100,5273,5590,290510,286956,6117496,0.952511615864
1512386040,5391,5860,290252,286336,6117496,0.952553789982
1512385980,5360,5720,290828,287420,6117496,0.952459633811
1512385920,5282,5600,291217,287692,6117496,0.952396045702
1512385860,5270,5730,291617,287932,6117496,0.952330659472
1512385800,5340,5840,290821,288064,6117496,0.95246077807


## stat table의 time 시작과 끝

In [19]:
%%bq query --name stat_unix_second
SELECT UNIX_SECONDS(time) as unix_second, stat_id, value
FROM `exem-191100.ods_im5.xapm_db_stat`
WHERE db_id = 6
GROUP BY time
ORDER BY time

In [20]:
%%bq sample --query stat_unix_second

unix_second,stat_id,value
1509721200,176,0
1509721200,317,0
1509721200,157,0
1509721200,93,18183782
1509721200,433,0
1509721200,240,0
1509721200,554,0
1509721200,525,0
1509721200,169,0
1509721200,132,0


In [229]:
# test 용

%%bq query --name stat_unix_second_test
SELECT UNIX_SECONDS(time) as unix_second, COUNT(stat_id) as count
FROM `exem-191100.ods_im5.xapm_db_stat`
WHERE db_id = 6
GROUP BY time
HAVING count <> 34
ORDER BY time

In [230]:
%%bq sample --query stat_unix_second_test

unix_second,count
1509721200,687
1509721260,687
1509721320,687
1509721380,687
1509721440,687
1509721500,687
1509721560,687
1509721620,687
1509721680,687
1509721740,687


In [215]:
stat_unix_second_sql = 'SELECT UNIX_SECONDS(time) as unix_second FROM `exem-191100.ods_im5.xapm_db_stat` WHERE db_id = 6 GROUP BY time ORDER BY time'

stat_unix_second_query = bq.Query(stat_unix_second_sql)

stat_unix_second_df = stat_unix_second_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [200]:
print(stat_unix_second_df.head(10))

   unix_second
0   1509721200
1   1509721260
2   1509721320
3   1509721380
4   1509721440
5   1509721500
6   1509721560
7   1509721620
8   1509721680
9   1509721740


In [26]:
%%bq query --name stat_unix_second_descending
SELECT UNIX_SECONDS(time) as unix_second
FROM `exem-191100.ods_im5.xapm_db_stat`
WHERE db_id = 6
GROUP BY time
ORDER BY time DESC

In [27]:
%%bq sample --query stat_unix_second_descending --count 1000

unix_second
1512386460
1512386400
1512386340
1512386280
1512386220
1512386160
1512386100
1512386040
1512385980
1512385920


## 맨 뒤의 2개의 timestamp는 stat table에만 있음을 알 수 있다. 시간은 unix_second를 기준으로 1,509,721,200 ~ 1,512,386,340 까지로 하면 좋을것 같다
''

## 이제 두 테이블에 모두 없는 빵꾸가 있는지 알아보자

''

- 데이터 프레임을 sql로 변환시키니깐 데이터가 하나도 안들어갔다, 왜 때문?

In [98]:
timerange_df = pd.DataFrame(columns=['A'])

for i in range(1509721200, 1512386340):
  if(i%60 == 0):
    timerange_df = timerange_df.append({'A':i}, ignore_index=True)
  
print(timerange_df.head(10))
print(timerange_df.shape[0])

              A
0  1.509721e+09
1  1.509721e+09
2  1.509721e+09
3  1.509721e+09
4  1.509721e+09
5  1.509722e+09
6  1.509722e+09
7  1.509722e+09
8  1.509722e+09
9  1.509722e+09
44419


In [64]:
empty_start_time = 0
empty_end_time = 0
prev = 1509721140
sum = 0

for i in range(stat_unix_second_df.shape[0]):
  now = stat_unix_second_df["unix_second"][i]
  if now-prev != 60:
    empty_start_time = prev
    empty_end_time = now
    print(empty_start_time, empty_end_time)
    sum += ( (empty_end_time-empty_start_time)/60 - 1)
  prev = stat_unix_second_df["unix_second"][i]
  
print(sum)

(1510218780, 1510218900)
(1510633920, 1510634340)
(1510747320, 1510747440)
(1510804380, 1510804560)
(1510828560, 1510828980)
(1510842480, 1510842900)
(1511240220, 1511240400)
(1511267220, 1511267460)
(1511409600, 1511409720)
(1511409960, 1511410380)
(1511410500, 1511410620)
(1511410620, 1511410980)
(1511411040, 1511411220)
(1511412300, 1511412420)
(1511412720, 1511412840)
(1511412960, 1511413260)
(1511413500, 1511413620)
(1511415360, 1511415540)
(1511415540, 1511415660)
(1511416140, 1511416260)
(1511416620, 1511416800)
(1511417820, 1511418900)
(1511421540, 1511421660)
(1511421660, 1511423700)
(1511436360, 1511436480)
(1511506380, 1511506560)
(1511510400, 1511510520)
(1511511240, 1511511360)
(1511881740, 1511881860)
(1511883120, 1511883240)
(1511883420, 1511883540)
114


In [99]:
os_stat_sql = 'SELECT time, cpu_usage, max_cpu_usage, free_memory, min_free_memory, total_memory, (total_memory - free_memory)/total_memory as use_memory_rate FROM `exem-191100.ods_im5.xapm_db_os_stat` WHERE db_id = 6 ORDER BY time'

os_stat_query = bq.Query(os_stat_sql)

os_stat_df = os_stat_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [202]:
os_stat_df.head(20)

,time,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate
0,2017-11-03 15:00:00,5244,6670,246592,241860,6117496,0.959691
1,2017-11-03 15:01:00,5201,5720,246007,241844,6117496,0.959786
2,2017-11-03 15:02:00,5168,5460,245579,241372,6117496,0.959856
3,2017-11-03 15:03:00,5157,5330,245377,241372,6117496,0.959889
4,2017-11-03 15:04:00,5263,5610,242621,240992,6117496,0.960340
5,2017-11-03 15:05:00,5220,5600,244343,240256,6117496,0.960058
6,2017-11-03 15:06:00,5218,5530,243530,239876,6117496,0.960191
7,2017-11-03 15:07:00,5274,5630,243200,239636,6117496,0.960245
8,2017-11-03 15:08:00,5142,5430,242762,239132,6117496,0.960317
9,2017-11-03 15:09:00,5260,5520,242303,238636,6117496,0.960392


In [58]:
print(os_stat_df.shape)

(44306, 7)


- 왜 시간이 15:00:00 으로 바꼈을까..?

In [20]:
%%bq query --name stat
SELECT time, stat_id, value
FROM `exem-191100.ods_im5.xapm_db_stat`
WHERE db_id = 6
ORDER BY time

In [21]:
%%bq sample --query stat

time,stat_id,value
2017-11-03 15:00:00,430,0
2017-11-03 15:00:00,41,0
2017-11-03 15:00:00,425,0
2017-11-03 15:00:00,442,0
2017-11-03 15:00:00,566,0
2017-11-03 15:00:00,579,0
2017-11-03 15:00:00,301,0
2017-11-03 15:00:00,151,1289
2017-11-03 15:00:00,494,0
2017-11-03 15:00:00,254,0


In [31]:
%%bq query --name stat_eliminatezero
SELECT stat_id
FROM(
  SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value
  FROM `exem-191100.ods_im5.xapm_db_stat`
  WHERE db_id = 6
  GROUP BY stat_id
  HAVING max_value <> 0  and min_value <> 0
  ORDER BY stat_id)


In [33]:
%%bq sample --query stat_eliminatezero --count 1000

stat_id
3
5
9
10
14
22
26
27
28
29


In [74]:
%%bq query --name stat
SELECT time, stat_id, value
FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat
WHERE db_id = 6 and EXISTS(
  SELECT stat_id
  FROM(
    SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value
    FROM `exem-191100.ods_im5.xapm_db_stat`
    WHERE db_id = 6
    GROUP BY stat_id
    HAVING max_value <> 0  and min_value <> 0
    ORDER BY stat_id) as t_nonzero
  WHERE t_stat.stat_id = t_nonzero.stat_id
  ) and stat_id<687
ORDER BY time, stat_id

In [75]:
%%bq sample --query stat --count 2000000

time,stat_id,value
2017-11-03 15:00:00,3,51
2017-11-03 15:00:00,5,86
2017-11-03 15:00:00,9,887656
2017-11-03 15:00:00,10,163
2017-11-03 15:00:00,14,2221
2017-11-03 15:00:00,22,1650312
2017-11-03 15:00:00,26,4662936
2017-11-03 15:00:00,27,926
2017-11-03 15:00:00,28,69869335
2017-11-03 15:00:00,29,0


# stat table 에서 time 당 유의미한 stat_id 의 개수가 34 가 아닌 놈은 하나도 없는것을 검증해보자

In [90]:
%%bq query --name stat_test
SELECT COUNT(time) as total_time
FROM(
  SELECT time, COUNT(stat_id) as count
  FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat
  WHERE db_id = 6 and EXISTS(
    SELECT stat_id
    FROM(
      SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value
      FROM `exem-191100.ods_im5.xapm_db_stat`
      WHERE db_id = 6
      GROUP BY stat_id
      HAVING max_value <> 0  and min_value <> 0
      ORDER BY stat_id) as t_nonzero
    WHERE t_stat.stat_id = t_nonzero.stat_id
    ) and stat_id<687
  GROUP BY time
  HAVING count <> 34
  )



In [91]:
%%bq sample --query stat_test

total_time
0


In [ ]:
%%bq query --name stat_unix_second
SELECT UNIX_SECONDS(time) as unix_second
FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat
WHERE db_id = 6 and EXISTS(
  SELECT stat_id
  FROM(
    SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value
    FROM `exem-191100.ods_im5.xapm_db_stat`
    WHERE db_id = 6
    GROUP BY stat_id
    HAVING max_value <> 0  and min_value <> 0
    ORDER BY stat_id) as t_nonzero
  WHERE t_stat.stat_id = t_nonzero.stat_id
  ) and stat_id<687
GROUP BY time
ORDER BY time

In [100]:
stat_unix_second_sql = 'SELECT UNIX_SECONDS(time) as unix_second FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat WHERE db_id = 6 and EXISTS( SELECT stat_id FROM( SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value FROM `exem-191100.ods_im5.xapm_db_stat` WHERE db_id = 6 GROUP BY stat_id HAVING max_value <> 0  and min_value <> 0 ORDER BY stat_id) as t_nonzero WHERE t_stat.stat_id = t_nonzero.stat_id ) and stat_id<687 GROUP BY time ORDER BY time'

stat_unix_second_query = bq.Query(stat_unix_second_sql)

stat_unix_second_df = stat_unix_second_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [232]:
print(stat_unix_second_query)
print(stat_unix_second_query.shape)

BigQuery Query - SELECT UNIX_SECONDS(time) as unix_second FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat WHERE db_id = 6 and EXISTS( SELECT stat_id FROM( SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value FROM `exem-191100.ods_im5.xapm_db_stat` WHERE db_id = 6 GROUP BY stat_id HAVING max_value <> 0  and min_value <> 0 ORDER BY stat_id) as t_nonzero WHERE t_stat.stat_id = t_nonzero.stat_id ) and stat_id<687 GROUP BY time ORDER BY time


AttributeError: 'Query' object has no attribute 'shape'

In [101]:
stat_sql = 'SELECT time, stat_id, value FROM `exem-191100.ods_im5.xapm_db_stat` as t_stat WHERE db_id = 6 and EXISTS( SELECT stat_id FROM( SELECT stat_id, MAX(value) as max_value, MIN(value) as min_value FROM `exem-191100.ods_im5.xapm_db_stat` WHERE db_id = 6 GROUP BY stat_id HAVING max_value <> 0  and min_value <> 0 ORDER BY stat_id) as t_nonzero WHERE t_stat.stat_id = t_nonzero.stat_id ) and stat_id<687 ORDER BY time, stat_id'

stat_query = bq.Query(stat_sql)

stat_df = stat_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [204]:
stat_df.head(40)

,time,stat_id,value
0,2017-11-03 15:00:00,3,51
1,2017-11-03 15:00:00,5,86
2,2017-11-03 15:00:00,9,887656
3,2017-11-03 15:00:00,10,163
4,2017-11-03 15:00:00,14,2221
5,2017-11-03 15:00:00,22,1650312
6,2017-11-03 15:00:00,26,4662936
7,2017-11-03 15:00:00,27,926
8,2017-11-03 15:00:00,28,69869335
9,2017-11-03 15:00:00,29,0


In [59]:
print(stat_df.shape)

(1506302, 3)


In [48]:
%%bq query --name wait
SELECT time, event_id, wait_time
FROM `exem-191100.ods_im5.xapm_db_wait`
WHERE db_id = 6
ORDER BY time

In [49]:
%%bq sample --query wait

time,event_id,wait_time
2017-11-03 15:00:00,278,0
2017-11-03 15:00:00,134,0
2017-11-03 15:00:00,143,0
2017-11-03 15:00:00,275,0
2017-11-03 15:00:00,399,0
2017-11-03 15:00:00,368,0
2017-11-03 15:00:00,1,300
2017-11-03 15:00:00,319,0
2017-11-03 15:00:00,1072,0
2017-11-03 15:00:00,607,0


In [116]:
df1 = pd.DataFrame([1000, 2000, 3000, 4000],[1, 2, 3, 4])
df2 = pd.DataFrame([1, 2, 3, 4])
df3 = pd.DataFrame(columns = ['A','B'])

In [117]:
m1 = df1.as_matrix()
m2 = df2.as_matrix()
m3 = df3.as_matrix()

In [102]:
timerange_arr = timerange_df.as_matrix()
print(timerange_arr)
print(timerange_arr.shape)

[[  1.50972120e+09]
 [  1.50972126e+09]
 [  1.50972132e+09]
 ..., 
 [  1.51238616e+09]
 [  1.51238622e+09]
 [  1.51238628e+09]]
(44419, 1)


In [103]:
os_stat_unix_second_arr = os_stat_unix_second_df.as_matrix()
print(os_stat_unix_second_arr)

[[1509721200]
 [1509721260]
 [1509721320]
 ..., 
 [1512386220]
 [1512386280]
 [1512386340]]


In [228]:
print(os_stat_unix_second_arr.shape)

(44306, 1)


In [104]:
stat_unix_second_arr = stat_unix_second_df.as_matrix()
print(stat_unix_second_arr)

[[1509721200]
 [1509721260]
 [1509721320]
 ..., 
 [1512386340]
 [1512386400]
 [1512386460]]


In [272]:
print(stat_unix_second_arr.shape)

(44303, 1)


In [105]:
stat_arr = stat_df.as_matrix()
os_stat_arr = os_stat_df.as_matrix()
print(stat_arr)
print(os_stat_arr)

[[Timestamp('2017-11-03 15:00:00') 3 51]
 [Timestamp('2017-11-03 15:00:00') 5 86]
 [Timestamp('2017-11-03 15:00:00') 9 887656]
 ..., 
 [Timestamp('2017-12-04 11:21:00') 680 55]
 [Timestamp('2017-12-04 11:21:00') 681 31]
 [Timestamp('2017-12-04 11:21:00') 683 55]]
[[Timestamp('2017-11-03 15:00:00') 5244 6670 ..., 241860 6117496
  0.9596906969779793]
 [Timestamp('2017-11-03 15:01:00') 5201 5720 ..., 241844 6117496
  0.9597863243392395]
 [Timestamp('2017-11-03 15:02:00') 5168 5460 ..., 241372 6117496
  0.9598562876052554]
 ..., 
 [Timestamp('2017-12-04 11:17:00') 5164 5410 ..., 286212 6117496
  0.9526263686972578]
 [Timestamp('2017-12-04 11:18:00') 5138 5430 ..., 285584 6117496
  0.9527183998158724]
 [Timestamp('2017-12-04 11:19:00') 5257 5690 ..., 285708 6117496
  0.9528313545280618]]


In [227]:
print(os_stat_arr.shape)
print(stat_arr.shape)

(44306, 7)
(1506302, 3)


In [106]:
join_arr = [[]]
emp = 0

for i in range(stat_unix_second_arr.shape[0]-2): #44303 -> 44301 (마지막 2개는 stat table에만 있으므로) #687이상의 인덱스를 가진 것들이 남아있었다ㅜㅜ
  stat_time = stat_unix_second_arr[i]
  os_stat_time = os_stat_unix_second_arr[i-emp] #44306 (44301과 5개의 차이 stat table에는 없는데 os table에는 있는 time이 5개..!)
  
  if(i>=42119):
    print(stat_time, os_stat_time)
    
  if(stat_time == os_stat_time):
    temp_arr = np.append([os_stat_time], [os_stat_arr[i-emp]], axis = 1) #44306
    
    for j in range(34): 
      temp_arr = np.append(temp_arr, [[stat_arr[i*34+j][2]]], axis=1) #44303
    
    if(i == 0):
      join_arr = temp_arr
    else:
      join_arr = np.append(join_arr, temp_arr, axis = 0)
  else:
    emp+=1

(array([1512255240]), array([1512255180]))
(array([1512255420]), array([1512255180]))
(array([1512255480]), array([1512255180]))
(array([1512255540]), array([1512255180]))
(array([1512255600]), array([1512255180]))
(array([1512255660]), array([1512255180]))
(array([1512255720]), array([1512255180]))
(array([1512255780]), array([1512255180]))
(array([1512255840]), array([1512255180]))
(array([1512255900]), array([1512255180]))
(array([1512255960]), array([1512255180]))
(array([1512256020]), array([1512255180]))
(array([1512256140]), array([1512255180]))
(array([1512256200]), array([1512255180]))
(array([1512256320]), array([1512255180]))
(array([1512256380]), array([1512255180]))
(array([1512256440]), array([1512255180]))
(array([1512256500]), array([1512255180]))
(array([1512256560]), array([1512255180]))
(array([1512256620]), array([1512255180]))
(array([1512256680]), array([1512255180]))
(array([1512256740]), array([1512255180]))
(array([1512256800]), array([1512255180]))
(array([151

In [292]:
print(join_arr)
print(join_arr.shape)

[[1509721200 Timestamp('2017-11-03 15:00:00') 5244 ..., 51 26 51]
 [1509721260 Timestamp('2017-11-03 15:01:00') 5201 ..., 51 26 51]
 [1509721320 Timestamp('2017-11-03 15:02:00') 5168 ..., 51 26 51]
 ..., 
 [1512255000 Timestamp('2017-12-02 22:50:00') 5305 ..., 56 31 56]
 [1512255060 Timestamp('2017-12-02 22:51:00') 5243 ..., 56 31 56]
 [1512255120 Timestamp('2017-12-02 22:52:00') 5264 ..., 56 31 56]]
(42119, 42)


In [289]:
print(os_stat_unix_second_arr[i-emp])

[1512255120]


In [290]:
temp_arr = np.append([os_stat_unix_second_arr[0]], [os_stat_arr[0]], axis = 1)
print(temp_arr)

[[1509721200 Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860
  6117496 0.9596906969779793]]


In [284]:
temp_arr = [os_stat_arr[0]]
print(temp_arr)

[array([Timestamp('2017-11-03 15:00:00'), 5244, 6670, 246592, 241860,
       6117496, 0.9596906969779793], dtype=object)]


In [262]:
#연습한거

print([os_stat_arr[0]])
print([[stat_arr[0][2]]])

temp_arr = np.append([os_stat_arr[0]], [[stat_arr[0][2]]], axis = 1)
temp_arr = np.append(temp_arr, [[stat_arr[0][2]]], axis = 1)

print(temp_arr)

join_arr4 = temp_arr
join_arr4 = np.append(join_arr4, temp_arr, axis =0)
print(join_arr4)
#join_arr4 = np.append(join_arr4, temp_arr,axis=0)
#join_arr4 = np.append(join_arr4, temp_arr,axis=0)
#join_arr4 = np.append(join_arr4, temp_arr,axis=0)

print(join_arr4)

[array([Timestamp('2017-11-03 15:00:00'), 5244, 6670, 246592, 241860,
       6117496, 0.9596906969779793], dtype=object)]
[[51]]
[[Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860 6117496
  0.9596906969779793 51 51]]
[[Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860 6117496
  0.9596906969779793 51 51]
 [Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860 6117496
  0.9596906969779793 51 51]]
[[Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860 6117496
  0.9596906969779793 51 51]
 [Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860 6117496
  0.9596906969779793 51 51]]


In [123]:
result_arr = [[]]
emp = 0

# 1512255120 = 2017-12-02 22:52:00

length = (1512255120-1509721200)/60 +1

for i in range(length):
  
  join_time = join_arr[i-emp][0]
  time = timerange_arr[i]
  
  if(time == join_time):
    if(time == 1509721200):
      result_arr = [np.append(timerange_arr[i], join_arr[i-emp])]
    else:  
      row_arr = [np.append(timerange_arr[i], join_arr[i-emp])]
      result_arr = np.append(result_arr, row_arr, axis = 0)
  else:
    emp+=1
    row_arr = [np.append(timerange_arr[i], join_arr[i-emp])]
    result_arr = np.append(result_arr, row_arr, axis = 0)

In [121]:
result_arr = [np.append(timerange_arr[0], join_arr[0])]
row_arr = [np.append(timerange_arr[0], join_arr[0])]
result_arr = np.append(result_arr, row_arr, axis = 0)
print(result_arr)

[[1509721200.0 1509721200 Timestamp('2017-11-03 15:00:00') 5244 6670 246592
  241860 6117496 0.9596906969779793 51 86 887656 163 2221 1650312 4662936
  926 69869335 0 0 170689 14 227787 473547 59 59 38 11145625 7 157952
  11303577 1960 1806 18183782 0 14952 249304 48 0 25 51 26 51]
 [1509721200.0 1509721200 Timestamp('2017-11-03 15:00:00') 5244 6670 246592
  241860 6117496 0.9596906969779793 51 86 887656 163 2221 1650312 4662936
  926 69869335 0 0 170689 14 227787 473547 59 59 38 11145625 7 157952
  11303577 1960 1806 18183782 0 14952 249304 48 0 25 51 26 51]]


In [124]:
print(result_arr)
print(result_arr.shape)

[[1509721200.0 1509721200 Timestamp('2017-11-03 15:00:00') ..., 51 26 51]
 [1509721260.0 1509721260 Timestamp('2017-11-03 15:01:00') ..., 51 26 51]
 [1509721320.0 1509721320 Timestamp('2017-11-03 15:02:00') ..., 51 26 51]
 ..., 
 [1512255000.0 1512255000 Timestamp('2017-12-02 22:50:00') ..., 56 31 56]
 [1512255060.0 1512255060 Timestamp('2017-12-02 22:51:00') ..., 56 31 56]
 [1512255120.0 1512255120 Timestamp('2017-12-02 22:52:00') ..., 56 31 56]]
(42233, 43)


In [126]:
result_df = pd.DataFrame(result_arr)

In [127]:
result_df.columns = ['UNIX_time','useless', 'timestamp','cpu_usage', 'max_cpu_usage', 'free_memory', 'min_free_memory', 'total_memory', 'use_memory_rate', 'stat_value_1', 'stat_value_2', 'stat_value_3', 'stat_value_4', 'stat_value_5', 'stat_value_6', 'stat_value_7', 'stat_value_8', 'stat_value_9', 'stat_value_10', 'stat_value_11', 'stat_value_12', 'stat_value_13', 'stat_value_14', 'stat_value_15', 'stat_value_16', 'stat_value_17', 'stat_value_18', 'stat_value_19', 'stat_value_20', 'stat_value_21', 'stat_value_22', 'stat_value_23', 'stat_value_24', 'stat_value_25', 'stat_value_26', 'stat_value_27', 'stat_value_28', 'stat_value_29', 'stat_value_30', 'stat_value_31', 'stat_value_32', 'stat_value_33', 'stat_value_34']

In [311]:
print(result_df)

         utf_time            timestamp cpu_usage max_cpu_usage free_memory  \
0      1509721200  2017-11-03 15:00:00      5244          6670      246592   
1      1509721260  2017-11-03 15:01:00      5201          5720      246007   
2      1509721320  2017-11-03 15:02:00      5168          5460      245579   
3      1509721380  2017-11-03 15:03:00      5157          5330      245377   
4      1509721440  2017-11-03 15:04:00      5263          5610      242621   
5      1509721500  2017-11-03 15:05:00      5220          5600      244343   
6      1509721560  2017-11-03 15:06:00      5218          5530      243530   
7      1509721620  2017-11-03 15:07:00      5274          5630      243200   
8      1509721680  2017-11-03 15:08:00      5142          5430      242762   
9      1509721740  2017-11-03 15:09:00      5260          5520      242303   
10     1509721800  2017-11-03 15:10:00      5197          5620      245299   
11     1509721860  2017-11-03 15:11:00      5278          5790  

In [299]:
#연습용

result_arr = [join_arr[0]]
result_arr = np.append(result_arr, [join_arr[1]], axis = 0)
result_arr = np.append(result_arr, [join_arr[2]], axis = 0)
print(result_arr)

[[1509721200 Timestamp('2017-11-03 15:00:00') 5244 6670 246592 241860
  6117496 0.9596906969779793 51 86 887656 163 2221 1650312 4662936 926
  69869335 0 0 170689 14 227787 473547 59 59 38 11145625 7 157952 11303577
  1960 1806 18183782 0 14952 249304 48 0 25 51 26 51]
 [1509721260 Timestamp('2017-11-03 15:01:00') 5201 5720 246007 241844
  6117496 0.9597863243392395 51 86 900186 163 23 1654267 4660257 908
  54933906 0 0 70131 14 139314 -93338 25 25 5 88883 2 39168 128051 21 16
  190873 0 269 18 3 0 25 51 26 51]
 [1509721320 Timestamp('2017-11-03 15:02:00') 5168 5460 245579 241372
  6117496 0.9598562876052554 51 86 872536 157 92 1618310 4623398 833
  145042783 0 0 168123 14 188466 381797 31 31 5 95846 3 43340 139187 77 32
  758169 0 6149 34 14 0 25 51 26 51]]


# 일단 빅쿼리에 저장시키자

In [130]:
result_df.to_gbq('ods_im5.os_and_stat2', 'exem-191100', if_exists = 'replace')




Streaming Insert is 23% Complete


GenericGBQException: Reason: badRequest, Message: Request payload size exceeds the limit: 10485760 bytes.

## 어제는 되고 오늘은 안되고... column 하나 추가한거에 대한 사이즈 증가분이 그렇게 critical 한가?

In [73]:
%%bq query --name os_and_stat
SELECT *
FROM `exem-191100.ods_im5.os_and_stat`
ORDER BY utf_time

In [74]:
%%bq sample --query os_and_stat

utf_time,timestamp,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate,stat_value_1,stat_value_2,stat_value_3,stat_value_4,stat_value_5,stat_value_6,stat_value_7,stat_value_8,stat_value_9,stat_value_10,stat_value_11,stat_value_12,stat_value_13,stat_value_14,stat_value_15,stat_value_16,stat_value_17,stat_value_18,stat_value_19,stat_value_20,stat_value_21,stat_value_22,stat_value_23,stat_value_24,stat_value_25,stat_value_26,stat_value_27,stat_value_28,stat_value_29,stat_value_30,stat_value_31,stat_value_32,stat_value_33,stat_value_34
1509721200,2017-11-03T15:00:00Z,5244,6670,246592,241860,6117496,0.95969069700000009,51,86,887656,163,2221,1650312,4662936,926,69869335,0,0,170689,14,227787,473547,59,59,38,11145625,7,157952,11303577,1960,1806,18183782,0,14952,249304,48,0,25,51,26,51
1509721260,2017-11-03T15:01:00Z,5201,5720,246007,241844,6117496,0.9597863243,51,86,900186,163,23,1654267,4660257,908,54933906,0,0,70131,14,139314,-93338,25,25,5,88883,2,39168,128051,21,16,190873,0,269,18,3,0,25,51,26,51
1509721320,2017-11-03T15:02:00Z,5168,5460,245579,241372,6117496,0.95985628760000008,51,86,872536,157,92,1618310,4623398,833,145042783,0,0,168123,14,188466,381797,31,31,5,95846,3,43340,139187,77,32,758169,0,6149,34,14,0,25,51,26,51
1509721380,2017-11-03T15:03:00Z,5157,5330,245377,241372,6117496,0.9598893077,51,86,895684,164,23,1630109,4633384,879,55032485,0,0,66856,14,119653,-63847,25,25,5,88473,2,39193,127667,21,16,189644,0,269,12,3,0,25,51,26,51
1509721440,2017-11-03T15:04:00Z,5263,5610,242621,240992,6117496,0.96033981880000008,51,86,940623,171,118,1705116,4710995,987,56625578,0,0,212743,14,258967,514557,31,31,7,122880,3,41881,164761,98,43,972390,0,6782,37,19,0,25,51,26,51
1509721500,2017-11-03T15:05:00Z,5220,5600,244343,240256,6117496,0.9600583311,51,86,904071,165,24,1663669,4770306,916,70332751,0,0,66856,14,142590,-83508,25,25,5,90112,25,330675,420787,22,17,202342,0,269,12,3,0,25,51,26,51
1509721560,2017-11-03T15:06:00Z,5218,5530,243530,239876,6117496,0.96019122860000006,51,86,919353,166,24,1659921,4662055,913,54609056,0,0,89781,14,54117,139314,25,25,5,88473,2,33868,122342,22,16,197836,0,276,12,3,0,25,51,26,51
1509721620,2017-11-03T15:07:00Z,5274,5630,243200,239636,6117496,0.9602451722,51,86,919368,169,25,1626453,4630817,915,55365525,0,0,66856,14,139314,60670,25,25,5,88473,3,40422,128896,23,18,207667,0,269,22,3,0,25,51,26,51
1509721680,2017-11-03T15:08:00Z,5142,5430,242762,239132,6117496,0.9603167701,51,86,897266,162,24,1665502,4667904,881,54634194,0,0,89781,14,67224,139314,25,25,5,90112,2,40448,130560,22,17,200294,0,276,12,3,0,25,51,26,51
1509721740,2017-11-03T15:09:00Z,5260,5520,242303,238636,6117496,0.96039180080000008,51,86,920192,164,23,1657372,4664063,911,55388670,0,0,66856,14,132760,60670,25,25,5,88473,2,36352,124825,21,16,189644,0,269,12,3,0,25,51,26,51


In [77]:
join1_sql = 'SELECT * FROM `exem-191100.ods_im5.os_and_stat`ORDER BY utf_time'

join1_query = bq.Query(join1_sql)

join1_df = join1_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [78]:
join1_arr = join1_df.as_matrix()
print(join1_arr)

[['1509721200' '2017-11-03T15:00:00Z' '5244' ..., '51' '26' '51']
 ['1509721260' '2017-11-03T15:01:00Z' '5201' ..., '51' '26' '51']
 ['1509721320' '2017-11-03T15:02:00Z' '5168' ..., '51' '26' '51']
 ..., 
 ['1512255000' '2017-12-02T22:50:00Z' '5305' ..., '56' '31' '56']
 ['1512255060' '2017-12-02T22:51:00Z' '5243' ..., '56' '31' '56']
 ['1512255120' '2017-12-02T22:52:00Z' '5264' ..., '56' '31' '56']]


In [17]:
%%bq query --name wait
SELECT UNIX_SECONDS(time) as unix_time, time, event_id, wait_time 
FROM `exem-191100.ods_im5.xapm_db_wait` as t_wait
WHERE db_id = 6 and EXISTS(
  SELECT *
  FROM(
    SELECT event_id, MIN(wait_time) as min_wait, MAX(wait_time) as max_wait
    FROM `exem-191100.ods_im5.xapm_db_wait`
    WHERE db_id = 6
    GROUP BY event_id
    HAVING ( min_wait <> 0 and max_wait <> 0)
    ) as t_comp
  WHERE t_wait.event_id = t_comp.event_id
  )
ORDER BY time, event_id

In [18]:
%%bq sample --query wait --count 1000

unix_time,time,event_id,wait_time
1509721200,2017-11-03 15:00:00,1,300
1509721200,2017-11-03 15:00:00,7,3286
1509721200,2017-11-03 15:00:00,74,595
1509721200,2017-11-03 15:00:00,259,300
1509721200,2017-11-03 15:00:00,289,173
1509721200,2017-11-03 15:00:00,304,300
1509721200,2017-11-03 15:00:00,312,300
1509721200,2017-11-03 15:00:00,354,607
1509721200,2017-11-03 15:00:00,378,96
1509721200,2017-11-03 15:00:00,379,320


In [19]:
wait_sql = 'SELECT UNIX_SECONDS(time) as unix_time, time, event_id, wait_time FROM `exem-191100.ods_im5.xapm_db_wait` as t_wait WHERE db_id = 6 and EXISTS( SELECT * FROM( SELECT event_id, MIN(wait_time) as min_wait, MAX(wait_time) as max_wait FROM `exem-191100.ods_im5.xapm_db_wait` WHERE db_id = 6 GROUP BY event_id HAVING ( min_wait <> 0 and max_wait <> 0) ) as t_comp WHERE t_wait.event_id = t_comp.event_id ) ORDER BY time, event_id'

wait_query = bq.Query(wait_sql)

wait_df = wait_query.execute(output_options = bq.QueryOutput.dataframe()).result()

In [20]:
wait_arr = wait_df.as_matrix()
print(wait_arr)

[[1509721200 Timestamp('2017-11-03 15:00:00') 1 300]
 [1509721200 Timestamp('2017-11-03 15:00:00') 7 3286]
 [1509721200 Timestamp('2017-11-03 15:00:00') 74 595]
 ..., 
 [1512391980 Timestamp('2017-12-04 12:53:00') 378 104]
 [1512391980 Timestamp('2017-12-04 12:53:00') 379 320]
 [1512391980 Timestamp('2017-12-04 12:53:00') 380 320]]


In [33]:
prev = 1509721140
empty = 0

for i in range(wait_arr.shape[0]):
  if(i%11 == 0):
    now = wait_arr[i][0]
    if( (now-prev) != 60):
      print(prev, now)
      empty += (now-prev)/60 - 1
    prev = now
print(empty)

(1509931800, 1509931863)
(1509931923, 1509931980)
(1510218840, 1510218960)
(1510353180, 1510353243)
(1510353243, 1510353300)
(1510633980, 1510634460)
(1510747380, 1510747500)
(1510804440, 1510804680)
(1510828620, 1510829040)
(1510842540, 1510842960)
(1511240280, 1511240460)
(1511267280, 1511267520)
(1511409660, 1511409780)
(1511410020, 1511410440)
(1511410560, 1511410680)
(1511410680, 1511411040)
(1511411100, 1511411280)
(1511412360, 1511412480)
(1511412780, 1511412900)
(1511413020, 1511413320)
(1511413560, 1511413680)
(1511415420, 1511415600)
(1511415600, 1511415720)
(1511415840, 1511415960)
(1511416200, 1511416320)
(1511416680, 1511416860)
(1511417880, 1511418960)
(1511421600, 1511421720)
(1511421720, 1511423760)
(1511436420, 1511436540)
(1511506440, 1511506620)
(1511510460, 1511510580)
(1511511300, 1511511420)
(1511881800, 1511881920)
(1511883180, 1511883300)
(1511883480, 1511883600)
(1511884620, 1511884740)
116


In [45]:
for i in range(wait_arr.shape[0]):
  if( (wait_arr[i][0])%60 != 0 ):
    wait_arr[i][0] = wait_arr[i][0]-3
    print(wait_arr[i][0])

1509931860
1509931860
1509931860
1509931860
1509931860
1509931860
1509931860
1509931860
1509931860
1509931860
1509931860
1509931920
1509931920
1509931920
1509931920
1509931920
1509931920
1509931920
1509931920
1509931920
1509931920
1509931920
1510353240
1510353240
1510353240
1510353240
1510353240
1510353240
1510353240
1510353240
1510353240
1510353240
1510353240


In [91]:
print(wait_arr.shape)
print(wait_arr.shape[0]/11)

(488356, 4)
44396


In [131]:
join1_arr = result_arr
join2_arr = [[]]
emp = 0
suc = 0

for i in range(int(join1_arr.shape[0])):
  join1_time = join1_arr[i][0]
  wait_time = wait_arr[(i-emp)*11][0]
  
  if(i<10000):
    print(int(join1_time), wait_time)
  if(int(join1_time) == wait_time):
    suc+=1
    
    temp_arr = [join1_arr[i]]
    
    for j in range(11): 
      temp_arr = np.append(temp_arr, [[wait_arr[(i-emp)*11+j][3]]], axis=1)
    
    if(i == 0):
      join2_arr = temp_arr
    else:
      join2_arr = np.append(join2_arr, temp_arr, axis = 0)
  else:
    emp+=1
    
print(suc)    

(1509721200, 1509721200)
(1509721260, 1509721260)
(1509721320, 1509721320)
(1509721380, 1509721380)
(1509721440, 1509721440)
(1509721500, 1509721500)
(1509721560, 1509721560)
(1509721620, 1509721620)
(1509721680, 1509721680)
(1509721740, 1509721740)
(1509721800, 1509721800)
(1509721860, 1509721860)
(1509721920, 1509721920)
(1509721980, 1509721980)
(1509722040, 1509722040)
(1509722100, 1509722100)
(1509722160, 1509722160)
(1509722220, 1509722220)
(1509722280, 1509722280)
(1509722340, 1509722340)
(1509722400, 1509722400)
(1509722460, 1509722460)
(1509722520, 1509722520)
(1509722580, 1509722580)
(1509722640, 1509722640)
(1509722700, 1509722700)
(1509722760, 1509722760)
(1509722820, 1509722820)
(1509722880, 1509722880)
(1509722940, 1509722940)
(1509723000, 1509723000)
(1509723060, 1509723060)
(1509723120, 1509723120)
(1509723180, 1509723180)
(1509723240, 1509723240)
(1509723300, 1509723300)
(1509723360, 1509723360)
(1509723420, 1509723420)
(1509723480, 1509723480)
(1509723540, 1509723540)


In [132]:
print(join2_arr)
print(join2_arr.shape)

[[1509721200.0 1509721200 Timestamp('2017-11-03 15:00:00') ..., 96 320 320]
 [1509721260.0 1509721260 Timestamp('2017-11-03 15:01:00') ..., 99 300 300]
 [1509721320.0 1509721320 Timestamp('2017-11-03 15:02:00') ..., 98 300 300]
 ..., 
 [1512255000.0 1512255000 Timestamp('2017-12-02 22:50:00') ..., 102 320 320]
 [1512255060.0 1512255060 Timestamp('2017-12-02 22:51:00') ..., 96 300 300]
 [1512255120.0 1512255120 Timestamp('2017-12-02 22:52:00') ..., 109 300 300]]
(42115, 54)


In [146]:
output_arr = [[]]

for i in range(join2_arr.shape[0]):
  sum_time = 0
  for j in range(11):
    sum_time += join2_arr[i][43+j]
  
  if(i==1):
    output_arr = np.append([join2_arr[i-1]], [[sum_time]], axis = 1)
  if(i>1):
    temp_arr = np.append([join2_arr[i-1]], [[sum_time]], axis = 1)
    output_arr = np.append(output_arr, temp_arr, axis = 0)
    

In [147]:
print(output_arr)
print(output_arr.shape)

[[1509721200.0 1509721200 Timestamp('2017-11-03 15:00:00') ..., 320 320
  6559]
 [1509721260.0 1509721260 Timestamp('2017-11-03 15:01:00') ..., 300 300
  6527]
 [1509721320.0 1509721320 Timestamp('2017-11-03 15:02:00') ..., 300 300
  15578]
 ..., 
 [1512254940.0 1512254940 Timestamp('2017-12-02 22:49:00') ..., 280 280
  7209]
 [1512255000.0 1512255000 Timestamp('2017-12-02 22:50:00') ..., 320 320
  7244]
 [1512255060.0 1512255060 Timestamp('2017-12-02 22:51:00') ..., 300 300
  7155]]
(42114, 55)


In [148]:
output_df = pd.DataFrame(output_arr)

output_df.columns = ['UNIX_time','useless', 'timestamp','cpu_usage', 'max_cpu_usage', 'free_memory', 'min_free_memory', 'total_memory', 'use_memory_rate', 'stat_value_1', 'stat_value_2', 'stat_value_3', 'stat_value_4', 'stat_value_5', 'stat_value_6', 'stat_value_7', 'stat_value_8', 'stat_value_9', 'stat_value_10', 'stat_value_11', 'stat_value_12', 'stat_value_13', 'stat_value_14', 'stat_value_15', 'stat_value_16', 'stat_value_17', 'stat_value_18', 'stat_value_19', 'stat_value_20', 'stat_value_21', 'stat_value_22', 'stat_value_23', 'stat_value_24', 'stat_value_25', 'stat_value_26', 'stat_value_27', 'stat_value_28', 'stat_value_29', 'stat_value_30', 'stat_value_31', 'stat_value_32', 'stat_value_33', 'stat_value_34', 'wait_event_1', 'wait_event_2','wait_event_3','wait_event_4','wait_event_5','wait_event_6','wait_event_7','wait_event_8','wait_event_9','wait_event_10','wait_event_11', 'Y_next_wait_sum']

In [151]:
output_df.to_gbq('ods_im5.train_1_im5_os_stat_wait', 'exem-191100', if_exists = 'replace', chunksize = 2000)




Streaming Insert is 4% Complete
Streaming Insert is 9% Complete
Streaming Insert is 14% Complete
Streaming Insert is 18% Complete
Streaming Insert is 23% Complete
Streaming Insert is 28% Complete
Streaming Insert is 33% Complete
Streaming Insert is 37% Complete
Streaming Insert is 42% Complete
Streaming Insert is 47% Complete
Streaming Insert is 52% Complete
Streaming Insert is 56% Complete
Streaming Insert is 61% Complete
Streaming Insert is 66% Complete
Streaming Insert is 71% Complete
Streaming Insert is 75% Complete
Streaming Insert is 80% Complete
Streaming Insert is 85% Complete
Streaming Insert is 90% Complete
Streaming Insert is 94% Complete
Streaming Insert is 99% Complete
Streaming Insert is 100% Complete




In [152]:
%%bq query --name train_1
SELECT *
FROM `exem-191100.ods_im5.train_1_im5_os_stat_wait`

In [153]:
%%bq sample --query train_1 --count 100

UNIX_time,useless,timestamp,cpu_usage,max_cpu_usage,free_memory,min_free_memory,total_memory,use_memory_rate,stat_value_1,stat_value_2,stat_value_3,stat_value_4,stat_value_5,stat_value_6,stat_value_7,stat_value_8,stat_value_9,stat_value_10,stat_value_11,stat_value_12,stat_value_13,stat_value_14,stat_value_15,stat_value_16,stat_value_17,stat_value_18,stat_value_19,stat_value_20,stat_value_21,stat_value_22,stat_value_23,stat_value_24,stat_value_25,stat_value_26,stat_value_27,stat_value_28,stat_value_29,stat_value_30,stat_value_31,stat_value_32,stat_value_33,stat_value_34,wait_event_1,wait_event_2,wait_event_3,wait_event_4,wait_event_5,wait_event_6,wait_event_7,wait_event_8,wait_event_9,wait_event_10,wait_event_11,Y_next_wait_sum
1512208320,1512208320,2017-12-02T09:52:00Z,5269,5580,240489,237124,6117496,0.9606883274,55,89,896635,165,28,1550280,1551947,960,54762675,0,0,-13087,14,-114788,-114788,22,22,5,88473,2,37324,125798,28,28,232652,0,39,7,4,0,24,55,31,55,300,3294,600,300,159,300,300,1203,100,280,280,7123
1509854220,1509854220,2017-11-05T03:57:00Z,5186,5410,231553,227992,6117496,0.9621490558,52,86,914174,163,23,1677985,4680168,849,55360886,0,0,66856,14,126206,-63847,25,25,5,85196,2,34688,119884,21,16,189235,0,191,12,3,0,26,52,26,52,300,3276,600,297,168,300,496,589,97,280,280,6608
1510607340,1510607340,2017-11-13T21:09:00Z,5487,6370,566237,562056,6117496,0.9074397433,38,55,887258,166,46,1557674,1686804,943,55023997,0,0,66856,14,132760,-86785,25,25,5,85196,2,35225,120422,43,39,384614,0,266,15,7,0,24,38,14,38,300,3281,595,300,155,300,573,1205,103,300,300,7160
1511449740,1511449740,2017-11-23T15:09:00Z,5166,5730,312408,308608,6117496,0.94893204670000009,52,84,917542,166,80,1645490,1653685,946,70415959,0,0,66856,14,149144,-67124,40,40,5,88473,6,45926,134400,50,26,663142,0,266,4,12,0,24,52,28,52,300,3301,600,300,161,300,300,906,103,280,280,6894
1511004180,1511004180,2017-11-18T11:23:00Z,5144,5540,280593,276784,6117496,0.954132704,52,82,896744,163,16,1607684,1618582,911,55438904,0,0,66856,14,139314,60670,25,25,5,85196,3,42470,127667,14,10,134348,0,269,1,2,0,25,52,27,52,300,3285,600,301,160,300,298,0,106,280,280,6002
1511013300,1511013300,2017-11-18T13:55:00Z,5187,5430,270328,266404,6117496,0.95581067810000009,52,82,929812,167,16,1674697,1686126,930,55422022,0,0,69511,14,139314,57394,26,26,8,135168,2,33868,169036,14,10,134348,0,269,1,3,0,25,52,27,52,300,3296,600,300,168,300,300,0,102,280,280,6007
1511385420,1511385420,2017-11-22T21:17:00Z,5137,5450,177692,173600,6117496,0.97095347510000007,51,82,900198,162,16,1621022,1627355,944,54639869,0,0,66856,14,126206,-63847,25,25,5,88473,2,37555,126028,14,10,135577,0,269,2,2,0,24,51,27,51,300,3294,600,280,163,300,495,0,105,320,320,5923
1511087820,1511087820,2017-11-19T10:37:00Z,5083,5500,291070,287552,6117496,0.9524200751,51,82,891029,162,16,1636904,1645331,933,55461049,0,0,66856,14,132760,60670,25,25,5,85196,2,35532,120729,14,10,137216,0,269,2,2,0,24,51,27,51,300,3301,600,300,163,300,300,0,105,280,280,6006
1511258700,1511258700,2017-11-21T10:05:00Z,5365,7210,289802,286500,6117496,0.9526273495,51,82,907732,166,18,1603917,1711453,918,70093827,0,0,53769,14,-14795,-178635,25,25,5,93388,25,294118,387507,16,11,151142,0,269,1,2,0,24,51,27,51,300,3286,600,300,158,300,430,0,102,320,320,5967
1511371740,1511371740,2017-11-22T17:29:00Z,5228,5550,148649,144636,6117496,0.97570100580000008,61,92,908779,168,18,1626233,1632222,963,55429497,0,0,66856,14,132760,-73678,25,25,5,88473,2,34278,122752,16,12,152780,0,269,11,2,0,24,61,37,61,300,3296,600,300,161,300,300,0,96,280,280,6003


In [60]:
test_arr = [[]]
temp_arr = [join1_arr[0]]
temp_arr = np.append(temp_arr, [[wait_arr[(0)*11+0][3]]], axis=1)

test_arr = temp_arr
test_arr = np.append(test_arr, temp_arr, axis = 0)
test_arr = np.append(test_arr, temp_arr, axis = 0)
print(test_arr)

[['1509952980' '2017-11-06T07:23:00Z' '5084' '5330' '280312' '276128'
  '6117496' '0.9541786378' '36' '56' '883985' '159' '23' '1672526'
  '1683315' '839' '54248646' '0' '0' '60313' '14' '52428' '-26214' '23'
  '23' '5' '93388' '4' '54809' '148198' '21' '17' '194969' '0' '265' '12'
  '3' '0' '23' '36' '13' '36' 300]
 ['1509952980' '2017-11-06T07:23:00Z' '5084' '5330' '280312' '276128'
  '6117496' '0.9541786378' '36' '56' '883985' '159' '23' '1672526'
  '1683315' '839' '54248646' '0' '0' '60313' '14' '52428' '-26214' '23'
  '23' '5' '93388' '4' '54809' '148198' '21' '17' '194969' '0' '265' '12'
  '3' '0' '23' '36' '13' '36' 300]
 ['1509952980' '2017-11-06T07:23:00Z' '5084' '5330' '280312' '276128'
  '6117496' '0.9541786378' '36' '56' '883985' '159' '23' '1672526'
  '1683315' '839' '54248646' '0' '0' '60313' '14' '52428' '-26214' '23'
  '23' '5' '93388' '4' '54809' '148198' '21' '17' '194969' '0' '265' '12'
  '3' '0' '23' '36' '13' '36' 300]]


In [ ]:
join2_arr = [[]]
emp = 0

# 1512255120 = 2017-12-02 22:52:00

length = (1512255120-1509721200)/60 +1

for i in range(join1_arr.shape[0]):
  
  wait_time = wait_arr[i-emp][0]
  time = join1_arr[i][0]
  
  if(time == wait_time):
    if(time == 1509721200):
      result_arr = [join_arr[i-emp]]
    else:  
      result_arr = np.append(result_arr, [join_arr[i-emp]], axis = 0)
  else:
    emp+=1
    result_arr = np.append(result_arr, [join_arr[i-emp]], axis = 0)

In [41]:
print(join1_arr.shape[0])
print(wait_arr.shape[0]/11)

42233
44396


In [44]:
print([join1_arr[0]])
temp_arr = np.append(temp_arr, [[stat_arr[i*34+j][2]]], axis=1)

[array(['1509952980', '2017-11-06T07:23:00Z', '5084', '5330', '280312',
       '276128', '6117496', '0.9541786378', '36', '56', '883985', '159',
       '23', '1672526', '1683315', '839', '54248646', '0', '0', '60313',
       '14', '52428', '-26214', '23', '23', '5', '93388', '4', '54809',
       '148198', '21', '17', '194969', '0', '265', '12', '3', '0', '23',
       '36', '13', '36'], dtype=object)]
